In [1]:
import rootutils
rootutils.setup_root('../', indicator=".project-root", pythonpath=True)
from src.model.model_module import LitMML
from src.model.utils import get_model_and_processor
from omegaconf import OmegaConf
import torch

In [3]:
ckpt_path = '/data/bhavin/ckpts/93t3xgrr/ckpt-epoch=14-loss-val=2.208.ckpt' # CLIP+ITM
model, processor = get_model_and_processor(config=OmegaConf.load('../configs/model/dual_encoder.yaml'))
lit_model = LitMML.load_from_checkpoint(ckpt_path, model=model, processor=processor)
model, processor = lit_model.model, lit_model.processor
itm_head = lit_model.itm_head
print(itm_head)

/home/phisch/venv_py3.8/py3.8/lib/python3.8/site-packages/lightning/pytorch/utilities/migration/utils.py:56: The loaded checkpoint was produced with Lightning v2.2.3, which is newer than your current Lightning version: v2.2.0.post0


Sequential(
  (0): Linear(in_features=1024, out_features=512, bias=True)
  (1): ReLU()
  (2): Linear(in_features=512, out_features=2, bias=True)
)


In [ ]:
random_tensor = torch.randn(4, 1024).to('cuda')
print("Input:")
print(random_tensor)
print("Output:")
print(itm_head(random_tensor))
# Logit 0: match, logit 1: no match

Input:
tensor([[-0.7769, -0.4848,  0.2617,  ..., -0.1786, -1.3835,  0.7025],
        [-1.0671, -1.0295, -0.5723,  ..., -0.0891, -2.1650,  0.7870],
        [-0.3240, -0.5818,  0.9855,  ...,  1.6382,  0.6030,  1.1252],
        [ 0.8481,  1.1996,  0.1406,  ...,  0.2321, -0.0255,  1.4783]],
       device='cuda:0')
Output:
tensor([[-51.1446,  50.4647],
        [-48.0244,  46.4656],
        [-33.9451,  33.9389],
        [-43.4788,  42.4063]], device='cuda:0', grad_fn=<AddmmBackward0>)


In [4]:
from src.data.data_module import MyDataModule
data_config = OmegaConf.load('../configs/data/test.yaml')
data_config.root = '/home/data'
data_config.dataloader.test.batch_size = 16
data_module = MyDataModule(data_config=data_config, processor=processor)
dataloaders = data_module.get_test_dataloaders()
print(dataloaders)

{'Caltech101': {'train': <torch.utils.data.dataloader.DataLoader object at 0x7f7bd9821fd0>, 'test': <torch.utils.data.dataloader.DataLoader object at 0x7f7a5d81d2b0>}, 'Caltech256': {'train': <torch.utils.data.dataloader.DataLoader object at 0x7f7a5d81d3d0>, 'test': <torch.utils.data.dataloader.DataLoader object at 0x7f7a5d81d4c0>}, 'CIFAR10': {'train': <torch.utils.data.dataloader.DataLoader object at 0x7f7a5d81d5b0>, 'test': <torch.utils.data.dataloader.DataLoader object at 0x7f7a5d81d6a0>}, 'CIFAR100': {'train': <torch.utils.data.dataloader.DataLoader object at 0x7f7a5d81d790>, 'test': <torch.utils.data.dataloader.DataLoader object at 0x7f7a5d81d880>}, 'DTD': {'train': <torch.utils.data.dataloader.DataLoader object at 0x7f7a5d81d970>, 'test': <torch.utils.data.dataloader.DataLoader object at 0x7f7a5d81da60>}, 'FGVCAircraft': {'train': <torch.utils.data.dataloader.DataLoader object at 0x7f7a5d81db50>, 'test': <torch.utils.data.dataloader.DataLoader object at 0x7f7a5d81dc40>}, 'Food10

Testing output of ITM head for one batch of images with same label template caption

In [ ]:
loader_caltech101 = dataloaders['Caltech101']['test']
batch = next(iter(loader_caltech101))
inputs, targets = batch
print(inputs.shape)
print(inputs)
print(targets.shape)
print(targets)

In [ ]:
features = model.get_image_features(inputs.to('cuda'))
print(features.shape)
print(features)

In [42]:
classnames = loader_caltech101.dataset.classnames
print(classnames)

['Faces', 'Faces', 'Leopards', 'Motorbikes', 'accordion', 'airplanes', 'anchor', 'ant', 'barrel', 'bass', 'beaver', 'binocular', 'bonsai', 'brain', 'brontosaurus', 'buddha', 'butterfly', 'camera', 'cannon', 'car side', 'ceiling fan', 'cellphone', 'chair', 'chandelier', 'cougar body', 'cougar face', 'crab', 'crayfish', 'crocodile', 'crocodile head', 'cup', 'dalmatian', 'dollar bill', 'dolphin', 'dragonfly', 'electric guitar', 'elephant', 'emu', 'euphonium', 'ewer', 'ferry', 'flamingo', 'flamingo head', 'garfield', 'gerenuk', 'gramophone', 'grand piano', 'hawksbill', 'headphone', 'hedgehog', 'helicopter', 'ibis', 'inline skate', 'joshua tree', 'kangaroo', 'ketch', 'lamp', 'laptop', 'llama', 'lobster', 'lotus', 'mandolin', 'mayfly', 'menorah', 'metronome', 'minaret', 'nautilus', 'octopus', 'okapi', 'pagoda', 'panda', 'pigeon', 'pizza', 'platypus', 'pyramid', 'revolver', 'rhino', 'rooster', 'saxophone', 'schooner', 'scissors', 'scorpion', 'sea horse', 'snoopy', 'soccer ball', 'stapler', 's

In [58]:
template = "a photo of a {}."
caption = template.format(classnames[33])
print(caption)

a photo of a dolphin.


In [59]:
caption_tokenized = processor(text=caption, padding=True, truncation=True, return_tensors="pt")['input_ids']
print(caption_tokenized)

tensor([[  101,  1037,  6302,  1997,  1037, 17801,  1012,   102]])


In [60]:
class_embedding = model.get_text_features(caption_tokenized.to('cuda'))
print(class_embedding.shape)
print(class_embedding)

torch.Size([1, 512])
tensor([[ 3.6345e-01, -9.5647e-02,  2.1639e-01, -6.4779e-01, -5.5535e-02,
          8.6517e-01,  7.8397e-01, -2.0864e-01, -1.1221e+00,  1.2738e+00,
          1.8853e+00,  2.2195e+00, -1.4295e-02, -1.2201e+00, -4.1211e-01,
          1.2364e+00,  5.6074e-01, -1.1828e+00, -6.3703e-01, -1.3100e+00,
         -1.0932e+00, -2.8891e-01, -1.1838e-01, -1.2127e-02,  2.3077e-01,
          6.1105e-02,  2.3475e-01, -1.6820e+00, -3.5458e-01, -5.6923e-02,
         -1.3380e+00,  1.0484e+00, -2.7518e-01, -4.9735e-01, -1.3581e-01,
          5.7183e-01,  1.2141e+00,  4.4431e-01,  1.4220e+00, -1.3342e+00,
         -3.0494e-01,  2.2135e-01, -8.9773e-02, -6.7024e-01, -7.0473e-01,
         -5.5582e-01, -1.5082e-01,  5.0931e-01,  3.6776e-01, -8.2197e-01,
         -1.0181e+00,  2.8512e-01,  3.1793e-01,  1.3616e+00, -1.4496e+00,
          3.7434e-01,  5.4244e-01, -4.7219e-01,  1.1753e+00, -6.1422e-01,
          1.3694e+00,  5.7788e-01,  8.1777e-01, -1.0191e+00,  1.5862e+00,
         -8.8899e

In [61]:
class_embedding = class_embedding.expand(features.shape[0], -1)
print(class_embedding.shape)
print(class_embedding)

torch.Size([128, 512])
tensor([[ 0.3634, -0.0956,  0.2164,  ..., -0.1207,  0.3989, -0.0211],
        [ 0.3634, -0.0956,  0.2164,  ..., -0.1207,  0.3989, -0.0211],
        [ 0.3634, -0.0956,  0.2164,  ..., -0.1207,  0.3989, -0.0211],
        ...,
        [ 0.3634, -0.0956,  0.2164,  ..., -0.1207,  0.3989, -0.0211],
        [ 0.3634, -0.0956,  0.2164,  ..., -0.1207,  0.3989, -0.0211],
        [ 0.3634, -0.0956,  0.2164,  ..., -0.1207,  0.3989, -0.0211]],
       device='cuda:0', grad_fn=<ExpandBackward0>)


In [62]:
multimodal_embeddings = torch.cat([features, class_embedding], dim=1)
print(multimodal_embeddings.shape)
print(multimodal_embeddings)

torch.Size([128, 1024])
tensor([[ 1.0409, -0.2451,  0.5526,  ..., -0.1207,  0.3989, -0.0211],
        [-0.0942,  0.4125, -0.3865,  ..., -0.1207,  0.3989, -0.0211],
        [ 1.0073, -0.3329,  0.7318,  ..., -0.1207,  0.3989, -0.0211],
        ...,
        [-0.9931, -0.2816,  0.8778,  ..., -0.1207,  0.3989, -0.0211],
        [ 0.1886, -0.9309,  0.2961,  ..., -0.1207,  0.3989, -0.0211],
        [ 0.0955, -1.4433,  0.4554,  ..., -0.1207,  0.3989, -0.0211]],
       device='cuda:0', grad_fn=<CatBackward0>)


In [63]:
logits = itm_head(multimodal_embeddings)
print(logits.shape)
print(logits)

torch.Size([128, 2])
tensor([[ -23.9402,   20.6330],
        [ -88.0289,   84.6446],
        [ -84.3488,   81.2620],
        [ -76.2346,   71.3858],
        [ -58.7587,   54.9962],
        [ -88.1227,   85.3095],
        [ -45.6891,   40.2724],
        [ -61.4424,   57.1874],
        [ -43.1788,   39.4497],
        [ -52.4088,   49.1902],
        [ -87.3254,   83.9368],
        [ -76.2545,   72.1617],
        [ -33.9653,   30.4254],
        [ -94.4177,   90.6792],
        [ -12.4889,    8.0885],
        [ -82.1984,   78.1978],
        [ -59.3414,   54.7147],
        [ -86.5210,   82.3090],
        [ -53.8556,   50.5758],
        [ -65.6672,   61.2014],
        [ -64.0128,   59.5802],
        [ -72.6029,   68.4713],
        [ -91.5395,   88.3638],
        [ -40.8095,   37.5870],
        [ -26.9941,   24.5121],
        [ -32.9176,   30.4817],
        [ -79.6010,   73.8596],
        [-109.4450,  104.2042],
        [ -66.2686,   63.2535],
        [ -85.3565,   81.8252],
        [ -44.1070,

In [66]:
where_class = targets == 33
print(where_class.nonzero())

tensor([[ 0],
        [12],
        [14],
        [87]])


In [69]:
print(torch.topk(logits[:, 0], k=5))

torch.return_types.topk(
values=tensor([-12.4889, -14.3789, -14.7361, -15.2405, -23.9402], device='cuda:0',
       grad_fn=<TopkBackward0>),
indices=tensor([ 14,  63, 113,  87,   0], device='cuda:0'))


3/5 output logits of neuron 0 ("match" neuron) with highest values are for matching image

In [5]:
loader_caltech101 = dataloaders['Caltech101']['test']
batch = next(iter(loader_caltech101))
inputs, targets = batch
print(inputs.shape)
print(inputs)
print(targets.shape)
print(targets)

torch.Size([16, 3, 224, 224])
tensor([[[[-0.6706, -0.6549, -0.6392,  ...,  0.2000,  0.2157,  0.2314],
          [-0.6627, -0.6392, -0.6235,  ...,  0.1843,  0.1843,  0.2000],
          [-0.6549, -0.6314, -0.6157,  ...,  0.1922,  0.2157,  0.2235],
          ...,
          [-0.7490, -0.6941, -0.6941,  ..., -0.4980, -0.5373, -0.4588],
          [-0.7569, -0.6784, -0.7098,  ..., -0.5373, -0.6157, -0.5529],
          [-0.6784, -0.7490, -0.6784,  ..., -0.5059, -0.6627, -0.6314]],

         [[-0.4980, -0.4824, -0.4667,  ..., -0.0745, -0.0745, -0.0745],
          [-0.4902, -0.4667, -0.4510,  ..., -0.0824, -0.0902, -0.0902],
          [-0.4824, -0.4588, -0.4431,  ..., -0.0353, -0.0431, -0.0353],
          ...,
          [-0.6784, -0.6235, -0.6235,  ..., -0.3412, -0.3961, -0.3333],
          [-0.6863, -0.6078, -0.6392,  ..., -0.3882, -0.4745, -0.4275],
          [-0.6078, -0.6784, -0.6078,  ..., -0.3569, -0.5137, -0.4824]],

         [[-0.0431, -0.0275, -0.0118,  ..., -0.3098, -0.3490, -0.3725],


In [9]:
feature = model.get_image_features(inputs.to('cuda'))[0]
print(feature.shape)
print(feature)
target = targets[0]
print(target)

torch.Size([512])
tensor([ 0.4966, -1.1076,  0.9623,  0.0328, -0.4631,  0.3592, -0.0867,  0.6580,
        -0.8098, -1.1734,  0.8335, -0.1524,  0.3226, -0.0650,  1.4281,  0.7274,
         1.3255,  1.4866,  0.4209, -0.3393, -0.9781,  0.2292,  0.4155,  0.6341,
        -0.1248,  0.7073,  0.4461,  0.4796,  1.7555,  0.6825, -1.4581, -0.6464,
         0.4240, -0.3164,  1.0878, -1.0156,  0.3479, -0.6284, -0.1435,  0.1391,
         0.4610,  0.5195, -0.6118, -0.9204, -1.8489,  0.6457,  1.0072,  0.5921,
         0.0798, -0.8742, -1.0587,  0.8590,  0.9960,  0.8942,  0.0062,  1.0705,
         0.6953, -0.5197, -0.7388,  0.7114, -0.2361,  0.0095,  0.6026,  0.7467,
        -0.0192,  0.4475, -0.4838,  0.4995,  0.3281,  0.3353, -0.5249, -0.0338,
         0.7304, -0.3361,  0.2528, -0.5073,  0.6438,  0.0041, -0.5131, -1.8558,
         0.0117,  0.1525, -0.4858, -0.9868, -0.5843,  1.4586,  0.5246, -0.4996,
         0.2811, -0.6550,  0.1762,  0.3072,  1.1519, -0.1377,  0.0911,  0.3932,
         0.4592, -0.20

In [13]:
classnames = loader_caltech101.dataset.classnames
classname_selection = classnames[target-3:target+3]
print(classname_selection)

['tick', 'trilobite', 'umbrella', 'watch', 'water lilly', 'wheelchair']


In [15]:
template = "a photo of a {}."
captions = [template.format(x) for x in classname_selection]
print(captions)

['a photo of a tick.', 'a photo of a trilobite.', 'a photo of a umbrella.', 'a photo of a watch.', 'a photo of a water lilly.', 'a photo of a wheelchair.']


In [16]:
caption_tokenized = processor(text=captions, padding=True, truncation=True, return_tensors="pt")['input_ids']
print(caption_tokenized)
class_embedding = model.get_text_features(caption_tokenized.to('cuda'))
print(class_embedding.shape)
print(class_embedding)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


tensor([[  101,  1037,  6302,  1997,  1037, 16356,  1012,   102,     0,     0,
             0],
        [  101,  1037,  6302,  1997,  1037, 13012,  4135, 16313,  2063,  1012,
           102],
        [  101,  1037,  6302,  1997,  1037, 12977,  1012,   102,     0,     0,
             0],
        [  101,  1037,  6302,  1997,  1037,  3422,  1012,   102,     0,     0,
             0],
        [  101,  1037,  6302,  1997,  1037,  2300, 14765,  1012,   102,     0,
             0],
        [  101,  1037,  6302,  1997,  1037, 13204,  1012,   102,     0,     0,
             0]])
torch.Size([6, 512])
tensor([[ 0.6263,  0.5572,  1.4229,  ...,  0.9261, -0.4038,  0.4696],
        [-0.1852,  0.1725,  1.2424,  ..., -0.1371, -1.1789, -0.7680],
        [ 0.2672, -0.2301,  2.2116,  ..., -0.6446, -0.3870, -0.8674],
        [ 0.3341,  0.3266,  0.7137,  ..., -0.0503, -0.2253, -0.1699],
        [ 0.0716,  0.1309,  1.6009,  ..., -0.0593, -0.4861, -0.9418],
        [ 0.3156,  0.5962,  1.3438,  ..., -0.4822, -

In [17]:
feature = feature.expand(class_embedding.shape[0], -1)
print(feature.shape)
print(feature)

torch.Size([6, 512])
tensor([[ 0.4966, -1.1076,  0.9623,  ..., -0.5224, -0.8284,  0.3881],
        [ 0.4966, -1.1076,  0.9623,  ..., -0.5224, -0.8284,  0.3881],
        [ 0.4966, -1.1076,  0.9623,  ..., -0.5224, -0.8284,  0.3881],
        [ 0.4966, -1.1076,  0.9623,  ..., -0.5224, -0.8284,  0.3881],
        [ 0.4966, -1.1076,  0.9623,  ..., -0.5224, -0.8284,  0.3881],
        [ 0.4966, -1.1076,  0.9623,  ..., -0.5224, -0.8284,  0.3881]],
       device='cuda:0', grad_fn=<ExpandBackward0>)


In [18]:
multimodal_embeddings = torch.cat([feature, class_embedding], dim=1)
print(multimodal_embeddings.shape)
print(multimodal_embeddings)

torch.Size([6, 1024])
tensor([[ 0.4966, -1.1076,  0.9623,  ...,  0.9261, -0.4038,  0.4696],
        [ 0.4966, -1.1076,  0.9623,  ..., -0.1371, -1.1789, -0.7680],
        [ 0.4966, -1.1076,  0.9623,  ..., -0.6446, -0.3870, -0.8674],
        [ 0.4966, -1.1076,  0.9623,  ..., -0.0503, -0.2253, -0.1699],
        [ 0.4966, -1.1076,  0.9623,  ..., -0.0593, -0.4861, -0.9418],
        [ 0.4966, -1.1076,  0.9623,  ..., -0.4822, -0.5651, -1.3828]],
       device='cuda:0', grad_fn=<CatBackward0>)


In [19]:
logits = itm_head(multimodal_embeddings)
print(logits.shape)
print(logits)

torch.Size([6, 2])
tensor([[-63.8308,  58.4294],
        [-48.5656,  45.5814],
        [-53.5590,  48.5309],
        [-19.3385,  13.9102],
        [-63.9262,  59.6700],
        [-43.9359,  36.5553]], device='cuda:0', grad_fn=<AddmmBackward0>)


In [ ]:
where_class = targets == 94
print(where_class.nonzero())
print(torch.topk(logits[:, 0], k=5))

Correct image/label embedding has highest logit value in neuron 0